In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Embedding, LSTM, Activation, Conv1D, MaxPooling1D
from keras.datasets import mnist
from keras.utils import np_utils, to_categorical
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 문장을 잘라줌.
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras_preprocessing.sequence import pad_sequences

from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# GPU 사용 초기화 및 할당.
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

import warnings
warnings.filterwarnings('ignore')

In [2]:
# 시드값 설정.
np.random.seed(3)
tf.random.set_seed(3)

# 크롤링을 통한 네이버 영화 평점 및 평가글 등 데이터.
df = pd.read_csv('data/naver_star_data.csv')
df.head()

# 평점 및 평가글만 가져옴.
df2 = df[['평점', '평가글']]

# 학습량이 너무 많아 우선 500개만 가져옴.
df2 = df2.sample(n=500).reset_index(drop=True)
df2.head()

,평점,평가글
0,10.0,재미있어요 마지막에는 슬퍼서 눙물
1,10.0,정말 재미있었음..ㅋ
2,10.0,안보면 후회합니다..
3,10.0,스토리 연기 다 좋았어요
4,10.0,진짜 배우분들 연기부터 음악까지.. 완벽했어요ㅠㅠ내일 시험인데 뿌리치고 엄마랑 영화...


In [3]:
#  결측치 확인.
df2.isna().sum()

평점      0
평가글    13
dtype: int64

In [4]:
# 결측치 제거.
df2.dropna(inplace=True)
df2.isna().sum()

평점     0
평가글    0
dtype: int64

In [5]:
# 글 인덱스를 평점을 기준으로 나눠서 추출.
a1 = df2.query('평점<=5').index
a2 = df2.query('평점> 5').index

# 새로운 값 설정.(1 : 긍정, 0 : 부정)
df2.loc[a1, '평점']=0
df2.loc[a2, '평점']=1

df2['평점'].value_counts()

1.0    415
0.0     72
Name: 평점, dtype: int64

In [6]:
# 데이터 분리.
docs    = df2['평가글'].values
classes = df2['평점'].values

# 단어 사전 생성.
token = Tokenizer()
token.fit_on_texts(docs)
# token.word_index

# 생성 단어사전 기준으로 단어들을 숫자로 변환.
x = token.texts_to_sequences(docs)

# 최대 단어 개수 구함.
max_cnt = 0
for c in x :
    a1 = len(c)
    if max_cnt < a1:
        max_cnt = a1
print(f'최대 단어 개수 : {max_cnt}개')              
print('-'*37)

# padding => 서로 길이가 다른 단어수(리스트)의 개수를 맞춰줌.
# 부족한 부분은  0으로 채움
pad_x = pad_sequences(x, max_cnt)
print(pad_x)

x_train, x_test, y_train, y_test = train_test_split(pad_x, classes, test_size=0.3, stratify=classes)
print('-'*37)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

# 단어의 개수에 1을 더한 값 구함.
word_size = len(token.word_index) + 1 
print('-'*37)
print(f'단어 개수 : {word_size}개')    

최대 단어 개수 : 35개
-------------------------------------
[[   0    0    0 ...  566  567  568]
 [   0    0    0 ...    3  569  111]
 [   0    0    0 ...    0  570  571]
 ...
 [   0    0    0 ... 3338   94  152]
 [   0    0    0 ... 3341 3342 3343]
 [   0    0    0 ...  565 3360 3361]]
-------------------------------------
(340, 35)
(340,)
(147, 35)
(147,)
-------------------------------------
단어 개수 : 3362개


In [7]:
# 모델 설정.
model = Sequential()

# 데이터 압축.
model.add( Embedding(word_size, 100) )

# 모델 세부 설정.
model.add(Dropout(0.5))
model.add(Conv1D(64,5,padding='valid', activation='relu', strides=1))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(55))
model.add(Dense(1))
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 100)         336200    
_________________________________________________________________
dropout (Dropout)            (None, None, 100)         0         
_________________________________________________________________
conv1d (Conv1D)              (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, None, 64)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 55)                26400     
_________________________________________________________________
dense (Dense)                (None, 1)                 56        
_________________________________________________________________
activation (Activation)      (None, 1)                 0

In [8]:
# 모델 컴파일.
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 학습.
model.fit(x_train, y_train, epochs=5, batch_size=100)

Epoch 1/5
4/4 [==============================] - 3s 30ms/step - loss: 0.6596 - accuracy: 0.8240
Epoch 2/5
4/4 [==============================] - 0s 8ms/step - loss: 0.5379 - accuracy: 0.8522
Epoch 3/5
4/4 [==============================] - 0s 8ms/step - loss: 0.4264 - accuracy: 0.8515
Epoch 4/5
4/4 [==============================] - 0s 7ms/step - loss: 0.4152 - accuracy: 0.8425
Epoch 5/5
4/4 [==============================] - 0s 7ms/step - loss: 0.3979 - accuracy: 0.8572


In [9]:
# 정확도 평가.
# RNN만 사용했을 경우와 비교했을 때, 작은 학습수임에도 불구하고 좋은 성능을 보임.
model.evaluate(x_test, y_test)[1]

5/5 [==============================] - 0s 7ms/step - loss: 0.4257 - accuracy: 0.8503


0.8503401279449463